# Create Missing Indexes

## Run the following query

Start with the defaults to look for the biggest impact

Adjust the paramters as need to additional performance needs

In [5]:
DECLARE @MinDBImpact int = 10,     -- > 10k high impact, look at > 1k if needed
		@DBName nvarchar(50) = null,  -- Filter by DB 
		@MinRowCount int = 100,     -- remove small tables from analysis
		@MinUserImpact int = 20       -- remove low impact indexes


SELECT Substring(statement, Charindex('.', statement) + 1, 99)        AS        TableName, 
       rc.[rowcount]                                                  AS        Row_Count, 
       CONVERT (DECIMAL (28, 1), migs.avg_total_user_cost * migs.avg_user_impact * (migs.user_seeks + migs.user_scans )) AS improvement_measure, 
       'CREATE INDEX IX_' 
       + Replace(Replace(Replace(Substring(statement, Charindex('.', statement, Charindex('.', statement)+1)+1, 99), '[', ''), ']', ''), ' ', '_') 
       + '_' 
       + Replace(Replace(Replace(Replace(Isnull(mid.equality_columns, inequality_columns), '[', ''), ']', ''), ' ', '_'), ',', '') 
       + ' ON ' 
       + Substring(statement, Charindex('.', statement)+1, 99) + ' (' + Isnull (mid.equality_columns, '') 
	   + CASE WHEN mid.equality_columns IS NOT NULL AND mid.inequality_columns IS NOT NULL 
		      THEN ',' ELSE '' END 
       + Isnull (mid.inequality_columns, '') + ')' 
       + Isnull (' INCLUDE (' + mid.included_columns + ')', '')       AS        create_index_statement, 
       migs.last_user_seek,
	   migs.avg_total_user_cost,
	   migs.avg_user_impact
FROM   sys.dm_db_missing_index_groups mig 
       INNER JOIN sys.dm_db_missing_index_group_stats migs 
               ON migs.group_handle = mig.index_group_handle 
       INNER JOIN sys.dm_db_missing_index_details mid 
               ON mig.index_handle = mid.index_handle 
       JOIN (SELECT Quotename(Schema_name(sOBJ.schema_id)) 
                    + '.' + Quotename(sOBJ.NAME) AS [TableName], 
                    Sum(sdmvPTNS.row_count)      AS [RowCount] 
             FROM   sys.objects AS sOBJ 
                    INNER JOIN sys.dm_db_partition_stats AS sdmvPTNS 
                            ON sOBJ.object_id = sdmvPTNS.object_id 
             WHERE  sOBJ.type = 'U' 
                    AND sOBJ.is_ms_shipped = 0x0 
                    AND sdmvPTNS.index_id < 2 
             GROUP  BY sOBJ.schema_id, 
                       sOBJ.NAME) rc 
         ON rc.tablename = Substring(statement, Charindex('.', statement) + 1, 99) 
WHERE  CONVERT (DECIMAL (28, 1), migs.avg_total_user_cost * migs.avg_user_impact * (migs.user_seeks + migs.user_scans )) > @MinDBImpact
       AND statement LIKE concat('%',@DBName, '%') -- filter by DB 
       AND avg_user_impact > @MinUserImpact -- filter by User Impact  
       AND rc.[rowcount] > @MinRowCount 
ORDER  BY Substring(statement, Charindex('.', statement) + 1, 99), 
          migs.avg_total_user_cost * migs.avg_user_impact * (migs.user_seeks + migs.user_scans ) DESC 

(2 rows affected)

Total execution time: 00:00:00.023

TableName,Row_Count,improvement_measure,create_index_statement,last_user_seek,avg_total_user_cost,avg_user_impact
[Person].[BusinessEntityAddress],19614,63.3,CREATE INDEX IX_BusinessEntityAddress_AddressTypeID ON [Person].[BusinessEntityAddress] ([AddressTypeID]),2020-07-24 12:02:21.830,0.859344507816509,24.57
[Sales].[SalesOrderHeader],31465,76.8,"CREATE INDEX IX_SalesOrderHeader_SalesPersonID ON [Sales].[SalesOrderHeader] ([SalesPersonID]) INCLUDE ([OrderDate], [SubTotal])",2020-07-24 12:02:18.577,0.8426846667099908,91.18


Group results by table starting with the highst impact

Look and script out existing indexes for the table

Compare suggestions and existing indexes. If there is overlap, merge the indexes. If the suggestion is to include most of the columns in the table, it is probably not worth including all columns.

Example:

Suggested index: CREATE INDEX IX_SalesOrderHeader_SalesPersonID ON [Sales].[SalesOrderHeader] ([SalesPersonID]) INCLUDE ([OrderDate], [SubTotal])

No index exists for SalesPersonID. So it is a good candidate to create. Copy the index to a new code box and run.



In [6]:
CREATE INDEX IX_SalesOrderHeader_SalesPersonID ON [Sales].[SalesOrderHeader] ([SalesPersonID]) INCLUDE ([OrderDate], [SubTotal])

Commands completed successfully.

Total execution time: 00:00:00.031